In [1]:
import time
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

In [2]:
# Insert your Spotify username and the credentials that you obtained from spotify developer
cid = '58ecd7aadf294b9aa038a3080ef670cb'
secret = '4a277ac2c0a744eea5c839b1ecb27002'
redirect_uri='http://localhost:7777/callback'
username = 'yvngflash_'

In [3]:
# Once the Authorisation is complete, we just need to `sp` to call the APIs
scope = 'user-top-read playlist-modify-private playlist-modify-public'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [4]:
# Getting top 1000 tracks from user
results = sp.current_user_top_tracks(limit=1000, offset=0,time_range='short_term')

In [5]:
# Convert it to Dataframe
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_favourite = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_favourite.head()

,track_name,album,track_id,artist,duration,popularity
0,Feels Like Death,Feels Like Death,03vMyCyCK7pVWjC1i1zur0,Levi Carter,237505,44
1,Deep End Freestyle,Sleepy Hallow Presents: Sleepy For President,2AlYncTpVHKwHb55F9lF6O,Sleepy Hallow,115200,75
2,Glory Boy,Freewave 3,5LpnrXjrt0BOU0iOGH78UN,LUCKI,111048,40
3,Place,Whole Lotta Red,1Bg2CNZw6S4e9cGWPmi0uI,Playboi Carti,117239,69
4,Faith,Faith,0TqNfrOY2IrpFRI2zxsMq4,LUCKI,107467,36


In [6]:
# Getting features for each song
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']] 
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness'],
                              features['mode']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness','duration_ms', 'key',
                                                             'valence', 'speechiness', 'mode'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

In [7]:
df_fav = fetch_audio_features(sp, df_favourite)
df_fav.head()

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness,mode
track_name,,,,,,,,,,,,,
Feels Like Death,03vMyCyCK7pVWjC1i1zur0,0.755,0.337000,0.626,110.088,0.0000,-6.247,0.1370,237505,6,0.465,0.293,1
Deep End Freestyle,2AlYncTpVHKwHb55F9lF6O,0.868,0.703000,0.479,133.928,0.0000,-6.687,0.1020,115200,4,0.776,0.445,0
Glory Boy,5LpnrXjrt0BOU0iOGH78UN,0.784,0.096600,0.519,129.953,0.0109,-9.868,0.0748,111048,1,0.599,0.096,1
Place,1Bg2CNZw6S4e9cGWPmi0uI,0.891,0.000674,0.628,131.060,0.0000,-6.350,0.1190,117240,1,0.040,0.105,1
Faith,0TqNfrOY2IrpFRI2zxsMq4,0.878,0.123000,0.584,125.069,0.0000,-10.014,0.1050,107468,10,0.488,0.419,1


### Getting the songs from all of Spotify's playlists

In [8]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [9]:
# Getting playlist IDs from each of Spotify's playlists
playlists = sp.user_playlists('spotify')
spotify_playlist_ids = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        spotify_playlist_ids.append(playlist['uri'][-22:])
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None
spotify_playlist_ids[:20]

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX1lVhptIYRda',
 '37i9dQZF1DX10zKzsJ2jva',
 '37i9dQZF1DX4JAvHpjipBk',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWXRqgorJj26U',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DWXJfnUiYjUKT',
 '37i9dQZF1DXcRXFNfZr7Tp',
 '37i9dQZF1DX4o1oenSJRJd',
 '37i9dQZF1DXbTxeAdrVG2l',
 '37i9dQZF1DX4UtSsGT1Sbe',
 '37i9dQZF1DWTJ7xPn4vNaz',
 '37i9dQZF1DXaKIA8E7WcJj',
 '37i9dQZF1DWSV3Tk4GO2fq',
 '37i9dQZF1DWTwnEm1IYyoj',
 '37i9dQZF1DX2A29LI7xHn1']

In [10]:
len(spotify_playlist_ids)

1398

### Getting tracks from Spotify playlists

In [11]:
# Getting the tracks in each playlist
def fetch_playlist_tracks(sp, playlistsid): 
    offset = 0
    tracks = []
    # Make the API request
    while True:
            content = sp.playlist_tracks( playlistsid, fields=None, limit=100, offset=offset, market=None)
            tracks += content['items']
        
            if content['next'] is not None:
                offset += 100
            else:
                break
    
    track_id = []
    track_name = []
    
    for track in tracks:
        track_id.append(track['track']['id'])
        track_name.append(track['track']['name'])
    
# Create the final df
    df_playlists_tracks = pd.DataFrame({"track_id":track_id, "track_name": track_name})
    return df_playlists_tracks

In [12]:
# Function to fetch the audio features from the songs of given playlist
def fetch_audio_features(sp, playlist_id):
    playlist = fetch_playlist_tracks(sp, playlist_id)
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness', 'duration_ms', 'key',
                                                             'valence', 'speechiness'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features


In [13]:
%%time
# This cell takes forever to run
# Build the dataframe froms the playlists
for i, playlist in enumerate(spotify_playlist_ids):
    try:
        string_command = "df_{} = fetch_audio_features(sp, playlist)".format(playlist)
        exec(string_command)
    except:
        pass


s(sp, playlist)
Create df_37i9dQZF1DXd9dy0tcoQft = fetch_audio_features(sp, playlist)
Create df_54LbSpjd64NjKPUyYAGkNU = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DX0ZgPbUQ205W = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DWTEbYi1zYy0Z = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DWXIuW81skHVz = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DWUgBy0IJPlHq = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DX5iKSDtVcf7Q = fetch_audio_features(sp, playlist)
Create df_4p3lupatTubTWpTgCZjgpk = fetch_audio_features(sp, playlist)
Create df_6tLfPD2P9syvBV8rgO2Yq1 = fetch_audio_features(sp, playlist)
Create df_3FfBohQXFlgFnYubzrvoq4 = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DXdgnLr18vPvu = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DX0MuOvUqmxDz = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DX2iUghHXGIjj = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DXcLDm348RRYK = fetch_audio_features(sp, playlist)
Crea

In [14]:
# To understand the playlist, it would be useful to get the average value for each audio feature
def fetch_audio_features_mean(sp, playlist_id):
    Playlist = fetch_audio_features(sp, playlist_id)
    return pd.DataFrame(Playlist.mean(), columns= [playlist_id])

In [15]:
%%time
# This cell also takes forever
# Merge all playlists together to get a large df of tracks
dataframes = []
# Loop through the filenames to populate dataframes with different dataframes 
for i in spotify_playlist_ids:
    try:
        dataframes.append(fetch_audio_features_mean(sp, i))
    except:
        pass

tify.com/v1/playlists/37i9dQZF1DWUI1rlvkdQnb/tracks returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DX4xuWVBs4FgJ/tracks returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DWWQp0YMTvpD3/tracks returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DWY8U6Zq7nvbE/tracks returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DWZIOAPKUdaKS/tracks returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DWX83CujKHHOn/tracks returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DX9nq0BqAtM4H/tracks returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DWUGsgkESc7qP/tracks returned 4

In [16]:
# Preparing dataset for model creation
X = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), dataframes)
X.head()

,37i9dQZF1DXcBWIGoYBM5M,37i9dQZF1DX0XUsuxWHRQd,37i9dQZF1DX1lVhptIYRda,37i9dQZF1DX10zKzsJ2jva,37i9dQZF1DX4JAvHpjipBk,37i9dQZF1DX4sWSpwq3LiO,37i9dQZF1DX4SBhb3fqCJd,37i9dQZF1DWXRqgorJj26U,37i9dQZF1DXcF6B6QPhFDv,37i9dQZF1DWXJfnUiYjUKT,...,37i9dQZF1DX60lVXkfYly8,37i9dQZF1DWSsCx004HXRd,37i9dQZF1DX5cO1uP1XC1g,37i9dQZF1DX8d1cgbQqKFy,37i9dQZF1DX6dCTsdzLHOw,37i9dQZF1DWWTuLtC3XxmA,37i9dQZF1DX2uk5oHRWimT,37i9dQZF1DWVzrEOEOM5Qu,7a80QjnKAMnZu5Lx50INeh,37i9dQZF1DX6afkUGYYDyK
danceability,0.691640,0.77362,0.588481,0.758680,0.647700,0.373895,0.612960,0.546407,0.474300,0.637646,...,0.454608,0.363178,0.682140,0.625951,0.364429,0.513623,0.485637,0.645667,0.546314,0.564891
acousticness,0.253529,0.13165,0.205147,0.218799,0.248206,0.991179,0.295725,0.191717,0.015792,0.252655,...,0.483768,0.697568,0.318963,0.897954,0.622196,0.492540,0.759558,0.649556,0.598711,0.472963
energy,0.605240,0.60698,0.697212,0.683540,0.594924,0.042993,0.549960,0.720138,0.834660,0.605640,...,0.210810,0.226596,0.699670,0.284082,0.342168,0.485785,0.285046,0.257219,0.448800,0.489473
tempo,117.936080,125.10542,117.451404,125.761740,120.881710,108.502914,118.395960,122.289014,135.988640,120.197162,...,89.633909,69.951770,114.143000,97.950785,123.038000,120.318331,117.683148,98.335250,113.116686,112.265055
instrumentalness,0.012115,0.01238,0.000105,0.017337,0.015423,0.927917,0.006264,0.046154,0.033967,0.015153,...,0.043920,0.006272,0.000797,0.003808,0.622893,0.002686,0.048426,0.020926,0.136469,0.064721


In [17]:
X = X.dropna()

In [18]:
Y = pd.DataFrame(df_fav.median(), columns= ['fav_playlist'])
Y = Y.drop('mode')
Y = Y.dropna()
Y.head()

,fav_playlist
danceability,0.772000
acousticness,0.084300
energy,0.481500
tempo,132.343500
instrumentalness,0.000003


In [19]:
# Turning Y into a scalar to pass through model
Y = Y.squeeze('columns')

In [20]:
Y = Y.sort_index( axis=0, level=None, ascending=True, inplace=False, kind='quicksort')
Y.head()

acousticness             0.084300
danceability             0.772000
duration_ms         132468.500000
energy                   0.481500
instrumentalness         0.000003
Name: fav_playlist, dtype: float64

## Model Building

In [21]:
# Analyze playlists as features
from sklearn.ensemble import RandomForestRegressor
# Can combine step above with this
forest = RandomForestRegressor(random_state=42, max_depth=5, max_features=10) 
forest.fit(X,Y)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
# Print the feature rankings
print("Playlist ranking:")
  
for f in range(len(importances)):
    print("%d. %s %f " % (f + 1, 
            X.columns[f], 
            importances[indices[f]]))

ValueError: Found array with 0 sample(s) (shape=(0, 868)) while a minimum of 1 is required.

In [22]:
frames = [df_37i9dQZF1DX4pUKG1kS0Ac, df_37i9dQZF1DXbYM3nMM0oPk, df_37i9dQZF1DWWBHeXOYZf74]
df = pd.concat(frames)
df = df.set_index(['track_id'])
df.tail()

,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_id,,,,,,,,,,,
2sijD9pZmS9D9s8hmQkQk5,0.619,0.80100,0.417,136.460,0.383000,-10.433,0.176,201187,5,0.196,0.0437
1ZFsgzYeQuF8IbQgmju5ap,0.643,0.23400,0.752,96.001,0.192000,-6.536,0.821,240053,2,0.582,0.0390
2T9EwC9mQUjaNlVtKCFDwX,0.614,0.28500,0.577,145.147,0.001620,-9.165,0.953,204000,5,0.253,0.0601
4k4mhm30I8pOn2k7O19pms,0.674,0.00754,0.796,80.033,0.000233,-6.236,0.156,135802,7,0.727,0.0545
2D2ZV5FGcEpAfc2rEMxhV2,0.798,0.17100,0.610,95.343,0.068800,-4.983,0.105,171425,4,0.520,0.0301


In [23]:
# Subtract mean of the favourite plalyist from the top 3 playlist 
df1 = df.subtract(Y, axis='columns') 
df1

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
track_id,,,,,,,,,,,
2aIdVb8v9KTpEZnftkz2mD,-0.07350,-0.321,95491.5,0.0685,-0.000003,-3.0,-0.0503,1.539,0.0430,-52.3425,0.198
3yNVfRQlUPViUh8O2V9SQn,0.03570,0.090,105798.5,0.1815,-0.000003,-4.0,0.1940,0.672,-0.0090,-47.0355,0.441
6o2g1BJvtYQssH84kBYs7y,-0.06880,0.102,133131.5,0.2325,-0.000003,3.0,-0.0272,3.012,-0.0900,-36.5175,0.145
0Uyh92tLyb9JawG8lmWCzJ,0.23370,-0.210,133064.5,0.1125,-0.000003,-1.0,0.1870,5.098,-0.1632,-69.4675,0.027
3jDdpx9PMlfMBS5tOBHFm9,-0.07799,-0.057,80624.5,0.3515,-0.000003,-2.0,0.0400,4.297,-0.1110,-36.8565,0.216
...,...,...,...,...,...,...,...,...,...,...,...
2sijD9pZmS9D9s8hmQkQk5,0.71670,-0.153,68718.5,-0.0645,0.382997,1.0,0.0520,-0.757,-0.1753,4.1165,-0.200
1ZFsgzYeQuF8IbQgmju5ap,0.14970,-0.129,107584.5,0.2705,0.191997,-2.0,0.6970,3.140,-0.1800,-36.3425,0.186
2T9EwC9mQUjaNlVtKCFDwX,0.20070,-0.158,71531.5,0.0955,0.001617,1.0,0.8290,0.511,-0.1589,12.8035,-0.143


In [24]:
df1 = df1.divide(Y, axis='columns') 
df1

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
track_id,,,,,,,,,,,
2aIdVb8v9KTpEZnftkz2mD,-0.871886,-0.415803,0.720862,0.142264,-1.000000,-0.75,-0.405645,-0.159053,0.196347,-0.395505,0.500000
3yNVfRQlUPViUh8O2V9SQn,0.423488,0.116580,0.798669,0.376947,-1.000000,-1.00,1.564516,-0.069450,-0.041096,-0.355405,1.113636
6o2g1BJvtYQssH84kBYs7y,-0.816133,0.132124,1.005005,0.482866,-1.000000,0.75,-0.219355,-0.311286,-0.410959,-0.275930,0.366162
0Uyh92tLyb9JawG8lmWCzJ,2.772242,-0.272021,1.004499,0.233645,-1.000000,-0.25,1.508065,-0.526871,-0.745205,-0.524903,0.068182
3jDdpx9PMlfMBS5tOBHFm9,-0.925148,-0.073834,0.608631,0.730010,-1.000000,-0.50,0.322581,-0.444088,-0.506849,-0.278491,0.545455
...,...,...,...,...,...,...,...,...,...,...,...
2sijD9pZmS9D9s8hmQkQk5,8.501779,-0.198187,0.518754,-0.133956,137274.985663,0.25,0.419355,0.078235,-0.800457,0.031105,-0.505051
1ZFsgzYeQuF8IbQgmju5ap,1.775801,-0.167098,0.812152,0.561786,68816.204301,-0.50,5.620968,-0.324514,-0.821918,-0.274607,0.469697
2T9EwC9mQUjaNlVtKCFDwX,2.380783,-0.204663,0.539989,0.198339,579.645161,0.25,6.685484,-0.052811,-0.725571,0.096744,-0.361111


In [25]:
# Add all the score 
df1['variation'] = df1.sum(axis=1)
# take the absoulte of the variatio
df1['variation'] = df1['variation'].abs()
df1

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,variation
track_id,,,,,,,,,,,,
2aIdVb8v9KTpEZnftkz2mD,-0.871886,-0.415803,0.720862,0.142264,-1.000000,-0.75,-0.405645,-0.159053,0.196347,-0.395505,0.500000,2.438420
3yNVfRQlUPViUh8O2V9SQn,0.423488,0.116580,0.798669,0.376947,-1.000000,-1.00,1.564516,-0.069450,-0.041096,-0.355405,1.113636,1.927886
6o2g1BJvtYQssH84kBYs7y,-0.816133,0.132124,1.005005,0.482866,-1.000000,0.75,-0.219355,-0.311286,-0.410959,-0.275930,0.366162,0.297505
0Uyh92tLyb9JawG8lmWCzJ,2.772242,-0.272021,1.004499,0.233645,-1.000000,-0.25,1.508065,-0.526871,-0.745205,-0.524903,0.068182,2.267633
3jDdpx9PMlfMBS5tOBHFm9,-0.925148,-0.073834,0.608631,0.730010,-1.000000,-0.50,0.322581,-0.444088,-0.506849,-0.278491,0.545455,1.521734
...,...,...,...,...,...,...,...,...,...,...,...,...
2sijD9pZmS9D9s8hmQkQk5,8.501779,-0.198187,0.518754,-0.133956,137274.985663,0.25,0.419355,0.078235,-0.800457,0.031105,-0.505051,137283.147240
1ZFsgzYeQuF8IbQgmju5ap,1.775801,-0.167098,0.812152,0.561786,68816.204301,-0.50,5.620968,-0.324514,-0.821918,-0.274607,0.469697,68823.356566
2T9EwC9mQUjaNlVtKCFDwX,2.380783,-0.204663,0.539989,0.198339,579.645161,0.25,6.685484,-0.052811,-0.725571,0.096744,-0.361111,588.452344


In [26]:
# Now we have the variation, we will take the songs with the least variation 
df2 = df1.nsmallest(50,'variation')
df2.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,variation
track_id,,,,,,,,,,,,
1Cv1YLb4q0RzL6pybtaMLo,1.170819,0.137306,0.197047,0.090343,-1.000000,0.25,-0.424194,-0.293923,-0.736073,-0.153551,0.752525,0.009702
1xTrwHzhHuNb5URVfuBp0f,-0.485172,-0.123057,0.588310,0.308411,-1.000000,0.00,0.895161,-0.236358,-0.479452,-0.244277,0.752525,0.023909
74DrA5fFoGSy4xgkZarZtP,1.894425,-0.655440,0.933841,-0.192108,-0.491039,0.00,-0.445161,0.414014,-0.860731,0.059689,-0.626263,0.031227
3eekarcy7kvN4yt5ZFzltW,-0.352313,-0.225389,0.326512,-0.113188,1.089606,0.75,0.693548,-0.094254,-0.855251,-0.422193,-0.847222,0.050144
1HwpWwa6bnqqRhK8agG4RS,-0.099644,-0.145078,0.617192,0.383178,-1.000000,1.25,-0.783871,-0.551261,-0.784475,-0.320140,1.356061,0.078039


In [27]:
def create_playlist(sp, username, playlist_name, playlist_description):
    playlists = sp.user_playlist_create(username, playlist_name, description = playlist_description)

In [28]:
create_playlist(sp, username, 'Your New Jams', 'This playlist was created using python!')

In [29]:
def fetch_playlists(sp, username):
    """
    Returns the user's playlists.
    """
        
    id = []
    name = []
    num_tracks = []
    
    # Make the API request
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        id.append(playlist['id'])
        name.append(playlist['name'])
        num_tracks.append(playlist['tracks']['total'])

    # Create the final df   
    df_playlists = pd.DataFrame({"id":id, "name": name, "#tracks": num_tracks})
    return df_playlists

In [30]:
fetch_playlists(sp,username).head()

,id,name,#tracks
0,3zdUVBgr30KKu1lFPimbRQ,Your New Jams,0
1,4OqhYtuar9Zcjj4djxIHMo,GOSPEL,34
2,3ALg99PJwQoQTZUgVOxZCr,LUCKI,38
3,4STXJNA3A9Hkz7soJiPz4s,POSITIVITY,77
4,1paxyb8f2gQW3ibSwMVend,AMATERASU 😈🔥,150


In [31]:
playlist_id = fetch_playlists(sp,username)['id'][0]

In [32]:
def enrich_playlist(sp, username, playlist_id, playlist_tracks):
    index = 0
    results = []
    
    while index < len(playlist_tracks):
        results += sp.user_playlist_add_tracks(username, playlist_id, tracks = playlist_tracks[index:index + 50])
        index += 50

In [33]:
list_track = df2.index
enrich_playlist(sp, username, playlist_id, list_track)
fetch_playlists(sp,username).head()

,id,name,#tracks
0,3zdUVBgr30KKu1lFPimbRQ,Your New Jams,50
1,4OqhYtuar9Zcjj4djxIHMo,GOSPEL,34
2,3ALg99PJwQoQTZUgVOxZCr,LUCKI,38
3,4STXJNA3A9Hkz7soJiPz4s,POSITIVITY,77
4,1paxyb8f2gQW3ibSwMVend,AMATERASU 😈🔥,150
